In [49]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, Float, Date,String
from sqlalchemy.orm import Session
import pandas as pd
import numpy as np

In [50]:
measure_dff=pd.read_csv("hawaii_measurements.csv")
station_df=pd.read_csv("hawaii_stations.csv")

In [51]:
cleanmeasure_df = measure_dff[np.isfinite(measure_dff['prcp'])]
cleanmeasure_df.to_csv("clean_measurements.csv", encoding='utf-8')
print(cleanmeasure_df.isnull().sum())

station    0
date       0
prcp       0
tobs       0
dtype: int64


In [52]:
measure_df=pd.read_csv("clean_measurements.csv")
station_df=pd.read_csv("hawaii_stations.csv")

In [53]:
measure_df.head(10)

,Unnamed: 0,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,5,USC00519397,2010-01-07,0.06,70
5,6,USC00519397,2010-01-08,0.00,64
6,7,USC00519397,2010-01-09,0.00,68
7,8,USC00519397,2010-01-10,0.00,73
8,9,USC00519397,2010-01-11,0.01,64
9,10,USC00519397,2010-01-12,0.00,61


In [54]:
station_df

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [74]:
Base = declarative_base()

engine = create_engine("sqlite:///Resources/Hawaii.sqlite",echo=False)
session = Session(engine)

In [75]:
class Measurement(Base):
    __tablename__="measurement"
    id = Column(Integer, primary_key=True)
    station=Column(String(255))
    date=Column(Date)
    prcp=Column(Float)
    tobs=Column(Float)

In [66]:
class Station(Base):
    __tablename__="station"
    ID = Column(Integer, primary_key=True)
    station=Column(String(255))
    name=Column(String(255))  
    latitude=Column(Float)
    longitude=Column(Float)
    elevation=Column(Float)

In [76]:
Base.metadata.create_all(engine)
measure_df.to_sql(con=engine,  name=Measurement.__tablename__, if_exists='replace')
station_df.to_sql(con=engine,  name=Station.__tablename__, if_exists='replace')



In [77]:
from sqlalchemy import  inspect
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [78]:
columns = inspector.get_columns('station')
for column in columns:
    print(column["name"], column["type"])

index BIGINT
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [79]:
Precipitation = Base.classes.measurement_coll

AttributeError: type object 'Base' has no attribute 'classes'

In [71]:
app = Flask(__name__)

NameError: name 'Flask' is not defined

In [ ]:
@app.route("/")
def welcome():
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations"
    )

@app.route("/api/v1.0/precipitation")
def weath():
    results=session.query(Precipitation.date).all()
    all_rain = list(np.ravel(results))
    return jsonify(all_rain)

@app.route("/api/v1.0/precipitation")
def temp():
    results=session.query(Precipitation.tobs).all()
    all_temp = list(np.ravel(results))
    return jsonify(all_temp)

@app.route("/api/v1.0/stations")
def stations():
    results=session.query(Precipitation.station).all()
    all_stations= list(np.ravel(results))
    return jsonify(all_stations)

@app.route("/api/v1.0/tobs")
    results=session.query(Precipitation.tobs).all()
    all_Obs=list(np.ravel(results))
    return jsonify(all_Obs)
    

    all_weath = []
    for weather in results:
        weather_dict = {}
        weather_dict ["date"] = weather.date
        weather_dict ["tobs"] = weather.tobs
        all_weath.append(weather_dict)
        return jsonify(all_weath)